In [8]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional

from nst_scraper import nst_on_ice_scraper
from db_utils import insert_player, update_player_db, check_last_update, append_player_ids

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)


In [9]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head()


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,20,388.516667,1,3,1,2,4,20.00,17,5.88,0.77,50,28,11,2,3,5,12,6,6,0,0,1,22,4,11,14,22,0,0,-
1,Brent Burns,CAR,D,18,294.850000,0,5,2,3,5,29.41,29,0.00,1.18,75,46,20,2,1,6,4,2,2,0,0,1,25,8,2,10,14,0,0,-
2,Corey Perry,EDM,R,20,204.600000,3,0,0,0,3,42.86,18,16.67,2.20,38,26,22,11,0,1,14,4,2,2,0,7,6,3,5,10,5,0,2,0.00
3,Alex Ovechkin,WSH,L,18,215.250000,9,7,4,3,16,72.73,34,26.47,3.27,69,49,40,16,0,5,6,3,3,0,0,0,7,4,41,8,3,0,0,-
4,Evgeni Malkin,PIT,C,21,316.166667,3,9,6,3,12,75.00,30,10.00,4.62,61,40,36,20,0,8,8,4,4,0,0,1,20,3,9,18,15,62,80,43.66


In [10]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='g')
goalie_stats_df.head()


,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,4,198.850000,92,85,7,0.924,2.11,1.08,5.51,12,9,3,0.750,0.91,-0.82,23,19,4,0.826,1.21,-1.39,51,51,0,1.000,0.00,1.71,6,10,39.33,20.14
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.58,8.91,27,27,0,1.000,0.00,4.91,32,31,1,0.969,0.27,2.63,47,47,0,1.000,0.00,1.58,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.34,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.61,23,23,0,1.000,0.00,0.77,3,8,30.81,17.20
3,Semyon Varlamov,NYI,8,405.900000,174,160,14,0.920,2.07,1.29,15.72,49,43,6,0.878,0.89,2.92,42,37,5,0.881,0.74,-0.23,67,64,3,0.955,0.44,-0.75,14,28,35.54,20.21
4,Jacob Markstrom,N.J,14,683.216667,317,288,29,0.909,2.55,-1.15,26.59,71,54,17,0.761,1.49,-4.08,88,80,8,0.909,0.70,1.98,138,134,4,0.971,0.35,0.63,41,59,39.39,21.07


In [11]:
from enum import Enum

class Position(Enum):
    C = 'C'
    L = 'L'
    R = 'R'
    D = 'D'
    G = 'G'
    
    @property
    def category(self) -> str:
        if self in {Position.C, Position.L, Position.R}:
            return 'F'
        elif self == Position.D:
            return 'D'
        elif self == Position.G:
            return 'G'
    
    def __str__(self) -> str:
        return self.value

In [12]:
from dataclasses import dataclass
from typing import Optional, Dict

@dataclass
class Player:
    name: str
    team: str
    position: Position
    player_id: Optional[int] = None

    def __str__(self) -> str:
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Converts the Player instance into a dictionary.
        
        Returns:
            Dict[str, Optional[str]]: A dictionary representation of the player.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position.value
        }

In [13]:
from typing import Union, Set

@dataclass
class Lineup:
    name: str
    forwards: List[Optional[Player]] = field(default_factory=lambda: [None] * 12)
    defense: List[Optional[Player]] = field(default_factory=lambda: [None] * 6)
    goalies: List[Optional[Player]] = field(default_factory=lambda: [None] * 2)
    
    ALLOWED_FORWARD_CATEGORIES = {'F'}
    ALLOWED_DEFENSE_CATEGORY = 'D'
    ALLOWED_GOALIE_CATEGORY = 'G'
    
    def __post_init__(self):
        self.validate_lineup_size()
    
    def validate_lineup_size(self):
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if total_players > 20:
            raise ValueError(f"Total number of players ({total_players}) exceeds the hard limit of 20.")
    
    def add_player(
        self,
        category: str,
        player: Player,
        slot: int,
        allowed_categories: Union[str, Set[str]],
        max_slots: int
    ):
        """
        Adds a player to the specified category and slot after validating their position category.
        Ensures that the total number of players does not exceed 20.
        """
        if isinstance(allowed_categories, str):
            allowed_categories = {allowed_categories}
        elif isinstance(allowed_categories, set):
            allowed_categories = allowed_categories
        else:
            raise TypeError("allowed_categories must be a string or a set of strings.")
        
        if player.position.category not in allowed_categories:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position.value}' "
                f"to {category}. Allowed categories: {', '.join(allowed_categories)}."
            )
        
        if not 0 <= slot < max_slots:
            raise IndexError(f"{category.capitalize()} slot must be between 0 and {max_slots - 1}.")
        
        current_category = getattr(self, category)
        if current_category[slot]:
            existing_player = current_category[slot].name
            print(f"Warning: Slot {slot + 1} in {category} is already occupied by '{existing_player}'. Overwriting.")
        
        # Check total players before adding
        total_players = sum(player is not None for player in self.forwards + self.defense + self.goalies)
        if current_category[slot] is None and total_players >= 20:
            raise ValueError("Cannot add more players. The lineup has reached the hard limit of 20 players.")
        
        current_category[slot] = player
        setattr(self, category, current_category)
        print(f"Added player '{player.name}' to {category[:-1].capitalize()} slot {slot + 1}.")
    
    def add_forward(self, player: Player, slot: int):
        self.add_player(
            category='forwards',
            player=player,
            slot=slot,
            allowed_categories=self.ALLOWED_FORWARD_CATEGORIES,
            max_slots=len(self.forwards)
        )
    
    def add_defense(self, player: Player, slot: int):
        self.add_player(
            category='defense',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_DEFENSE_CATEGORY},
            max_slots=len(self.defense)
        )
    
    def set_goalie(self, player: Player, slot: int):
        self.add_player(
            category='goalies',
            player=player,
            slot=slot,
            allowed_categories={self.ALLOWED_GOALIE_CATEGORY},
            max_slots=len(self.goalies)
        )
    
    def adjust_slots(self, category: str, delta: int):
        """
        Adjusts the number of slots in the specified category by delta.
        Allows ±1 adjustment only.
        
        Args:
            category (str): The category to adjust ('forwards' or 'defense').
            delta (int): The change in number of slots (+1 or -1).
        """
        if category not in {'forwards', 'defense'}:
            raise ValueError("Can only adjust 'forwards' or 'defense' categories.")
        if delta not in {-1, 1}:
            raise ValueError("Delta must be either +1 or -1.")
        
        current_slots = getattr(self, category)
        new_slot_count = len(current_slots) + delta
        
        if category == 'forwards':
            if not (11 <= new_slot_count <= 13):
                raise ValueError("Number of forwards can only vary by ±1 from the default of 12.")
        elif category == 'defense':
            if not (5 <= new_slot_count <= 7):
                raise ValueError("Number of defensemen can only vary by ±1 from the default of 6.")
        
        if delta == 1:
            current_slots.append(None)
        elif delta == -1:
            removed_player = current_slots.pop()
            if removed_player:
                print(f"Removed player '{removed_player.name}' from {category}.")
        
        setattr(self, category, current_slots)
        print(f"Adjusted {category} slots to {len(getattr(self, category))}.")
        self.validate_lineup_size()
    
    def display_lineup(self):
        """
        Prints the current lineup in a structured format.
        """
        print(f"Lineup: {self.name}\n")
        
        for category, title in [('forwards', 'Forwards'), ('defense', 'Defense'), ('goalies', 'Goalies')]:
            print(f"{title}:")
            for idx, player in enumerate(getattr(self, category), start=1):
                player_info = str(player) if player else 'Empty'
                print(f"  Slot {idx}: {player_info}")
            print()
    
    def to_dataframe(self) -> pd.DataFrame:
        """
        Converts the lineup into a pandas DataFrame.
        """
        data = []
        for category, pos in [('forwards', 'f'), ('defense', 'd'), ('goalies', 'g')]:
            for idx, player in enumerate(getattr(self, category), start=1):
                data.append({
                    'Position': f"{pos}{idx}",
                    'Player': player.name if player else 'Empty'
                })
        return pd.DataFrame(data)
    
    def to_transposed_dataframe(self) -> pd.DataFrame:
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        and the row contains the corresponding player names.
        """
        df = self.to_dataframe()
        transposed_data = {f"{row['Position']}": row['Player'] for _, row in df.iterrows()}
        return pd.DataFrame([transposed_data])

In [14]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    try:
        position_enum = Position(row['position'])  # Convert abbreviation to Position Enum
    except ValueError:
        print(f"Invalid position '{row['position']}' for player '{row['player']}'. Skipping.")
        continue
    
    player = Player(
        name=row['player'],
        team=row['team'],
        position=position_enum
        # player_id is not set initially
    )
    player_list.append(player)

# Convert player list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in player_list]).head()

Invalid position 'C, R' for player 'Gustav Nyquist'. Skipping.
Invalid position 'C, L' for player 'Jeff Skinner'. Skipping.
Invalid position 'C, L' for player 'Mark Jankowski'. Skipping.
Invalid position 'C, R' for player 'Michael McCarron'. Skipping.
Invalid position 'C, R' for player 'Philip Tomasino'. Skipping.
Invalid position 'L, R' for player 'Cole Smith'. Skipping.


,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [15]:
goalie_stats_df.head()

,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,4,198.850000,92,85,7,0.924,2.11,1.08,5.51,12,9,3,0.750,0.91,-0.82,23,19,4,0.826,1.21,-1.39,51,51,0,1.000,0.00,1.71,6,10,39.33,20.14
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.58,8.91,27,27,0,1.000,0.00,4.91,32,31,1,0.969,0.27,2.63,47,47,0,1.000,0.00,1.58,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.34,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.61,23,23,0,1.000,0.00,0.77,3,8,30.81,17.20
3,Semyon Varlamov,NYI,8,405.900000,174,160,14,0.920,2.07,1.29,15.72,49,43,6,0.878,0.89,2.92,42,37,5,0.881,0.74,-0.23,67,64,3,0.955,0.44,-0.75,14,28,35.54,20.21
4,Jacob Markstrom,N.J,14,683.216667,317,288,29,0.909,2.55,-1.15,26.59,71,54,17,0.761,1.49,-4.08,88,80,8,0.909,0.70,1.98,138,134,4,0.971,0.35,0.63,41,59,39.39,21.07


In [27]:
# Creating Player instances from the goalie_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=Position.G  # Set position to 'G' for goalies
        # player_id is not set initially
    )
    goalie_list.append(player)

# Convert goalie list to DataFrame to verify
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position.value
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,ANA,G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [17]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-11-19'

In [18]:
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [19]:
# Check the last update time of the players database
last_update = check_last_update(db_config)

INFO:db_utils:Database connection established.
INFO:db_utils:Last database update was on: 2024-11-20
INFO:db_utils:Database connection closed.


In [20]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_config, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-11-20


In [21]:
def add_player_to_lineup(lineup: Lineup, player: Player, category: str):
    """
    Adds a player to the lineup in the specified category, handling potential errors.
    
    Args:
        lineup (Lineup): The lineup object.
        player (Player): The player to add.
        category (str): The category ('forwards', 'defense', 'goalies').
    """
    try:
        if category == 'forwards':
            slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
            lineup.add_forward(player, slot)
        elif category == 'defense':
            slot = next(i for i, p in enumerate(lineup.defense) if p is None)
            lineup.add_defense(player, slot)
        elif category == 'goalies':
            slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, slot)
        else:
            print(f"Unknown category '{category}'.")
    except StopIteration:
        print(f"No available slots to add player '{player.name}' in category '{category}'.")
    except ValueError as ve:
        print(ve)
    except IndexError as ie:
        print(ie)

In [28]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team):
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Added player 'Ryan Reaves' to Forward slot 1.
Added player 'Max Pacioretty' to Forward slot 2.
Added player 'John Tavares' to Forward slot 3.
Skipping player 'Oliver Ekman-Larsson': Cannot add player 'Oliver Ekman-Larsson' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Chris Tanev': Cannot add player 'Chris Tanev' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jani Hakanpää': Cannot add player 'Jani Hakanpää' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed categories: F.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed categories: F.
Added player 'Max Domi' to Forward slot 4.
Added player 'William Nylander' to Forward slot 5.
Added player 'Mitch Marner' to Forward slot 6.
Added player 'Steven Lorentz' to Forward slot 7.
Skipping player 'Philippe Myers': Cannot add player 'Phil

In [25]:
def create_flexible_lineup(team: str) -> Lineup:
    """
    Creates a flexible lineup for the specified team, allowing for ±1 forward or defense spot.
    
    Args:
        team (str): The team name to filter players.
    
    Returns:
        Lineup: The configured lineup object.
    """
    lineup = Lineup(f"Flexible Lineup for {team}")
    
    # Example logic to adjust slots based on team strategy
    # Here, we simply allow flexibility; implement specific rules as needed
    allow_extra_forward = True  # Example condition
    allow_extra_defense = False  # Example condition
    
    if allow_extra_forward:
        lineup.adjust_slots('forwards', 1)  # Increase forwards to 13
    elif allow_extra_defense:
        lineup.adjust_slots('defense', 1)  # Increase defense to 7
    
    # Add players to the lineup
    forward_count = 0
    defense_count = 0
    goalie_count = 0
    
    for player in player_list:
        if player.team == team:
            if player.position.category == 'F' and forward_count < len(lineup.forwards):
                try:
                    lineup.add_forward(player, forward_count)
                    forward_count += 1
                except ValueError:
                    continue
            elif player.position.category == 'D' and defense_count < len(lineup.defense):
                try:
                    lineup.add_defense(player, defense_count)
                    defense_count += 1
                except ValueError:
                    continue
    
    for goalie in goalie_list:
        if goalie.team == team and goalie_count < len(lineup.goalies):
            try:
                lineup.set_goalie(goalie, goalie_count)
                goalie_count += 1
            except ValueError:
                continue
    
    return lineup

# Create and display a flexible lineup for 'TOR'
flexible_lineup = create_flexible_lineup('TOR')
flexible_lineup.display_lineup()

Adjusted forwards slots to 13.
Added player 'Ryan Reaves' to Forward slot 1.
Added player 'Max Pacioretty' to Forward slot 2.
Added player 'John Tavares' to Forward slot 3.
Added player 'Oliver Ekman-Larsson' to Defens slot 1.
Added player 'Chris Tanev' to Defens slot 2.
Added player 'Jani Hakanpää' to Defens slot 3.
Added player 'Morgan Rielly' to Defens slot 4.
Added player 'Jake McCabe' to Defens slot 5.
Added player 'Max Domi' to Forward slot 4.
Added player 'William Nylander' to Forward slot 5.
Added player 'Mitch Marner' to Forward slot 6.
Added player 'Steven Lorentz' to Forward slot 7.
Added player 'Philippe Myers' to Defens slot 6.
Added player 'Auston Matthews' to Forward slot 8.
Added player 'David Kampf' to Forward slot 9.
Added player 'Connor Dewar' to Forward slot 10.
Added player 'Pontus Holmberg' to Forward slot 11.
Added player 'Nicholas Robertson' to Forward slot 12.
Added player 'Bobby McMann' to Forward slot 13.
Lineup: Flexible Lineup for TOR

Forwards:
  Slot 1: R

In [29]:
my_lineup.to_dataframe()

,Position,Player
0,f1,Ryan Reaves
1,f2,Max Pacioretty
2,f3,John Tavares
3,f4,Max Domi
4,f5,William Nylander
5,f6,Mitch Marner
6,f7,Steven Lorentz
7,f8,Auston Matthews
8,f9,David Kampf
9,f10,Connor Dewar


In [24]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Empty,Empty


In [15]:
def assign_player_ids_to_lineup(transposed_lineup_df, db_config):
    """
    Processes the transposed_lineup_df DataFrame to append player IDs next to each player's name.
    
    This function performs the following steps:
        1. Extracts all unique player names from the lineup DataFrame.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Inserts new columns into the DataFrame with the corresponding player IDs.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.
    
    Returns:
        pd.DataFrame: The updated DataFrame with player IDs added.
    """
    import pandas as pd

    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")

    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()

    # Create Player instances
    player_list = []
    for name in player_names:
        if name != 'Empty':
            player = Player(name=name, team=None, position=None)  # Team and position can be set if available
            player_list.append(player)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_config)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()

    # Insert player_id columns adjacent to each player name column
    for col in player_columns:
        player_name = updated_df.at[0, col]
        if player_name != 'Empty':
            player_id = name_to_id.get(player_name, None)
            id_column = f"{col}_id"
            updated_df[id_column] = player_id
        else:
            id_column = f"{col}_id"
            updated_df[id_column] = None  # Assign None for empty slots

    return updated_df

In [16]:
# Assuming you have already created `transposed_lineup_df` and `db_config`
updated_lineup_df = assign_player_ids_to_lineup(transposed_lineup_df, db_config)

# Display the updated DataFrame
updated_lineup_df

INFO:db_utils:Database connection established.
INFO:db_utils:Assigned player_id 8471817 to Ryan Reaves.
INFO:db_utils:Assigned player_id 8474157 to Max Pacioretty.
INFO:db_utils:Assigned player_id 8475166 to John Tavares.
INFO:db_utils:Assigned player_id 8477503 to Max Domi.
INFO:db_utils:Assigned player_id 8477939 to William Nylander.
INFO:db_utils:Assigned player_id 8478483 to Mitch Marner.
INFO:db_utils:Assigned player_id 8478904 to Steven Lorentz.
INFO:db_utils:Assigned player_id 8479318 to Auston Matthews.
INFO:db_utils:Assigned player_id 8480144 to David Kampf.
INFO:db_utils:Assigned player_id 8480980 to Connor Dewar.
INFO:db_utils:Assigned player_id 8480995 to Pontus Holmberg.
INFO:db_utils:Assigned player_id 8481582 to Nicholas Robertson.
INFO:db_utils:Assigned player_id 8475171 to Oliver Ekman-Larsson.
INFO:db_utils:Assigned player_id 8475690 to Chris Tanev.
INFO:db_utils:Assigned player_id 8475825 to Jani Hakanpää.
INFO:db_utils:Assigned player_id 8476853 to Morgan Rielly.
IN

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_id,f2_id,f3_id,f4_id,f5_id,f6_id,f7_id,f8_id,f9_id,f10_id,f11_id,f12_id,d1_id,d2_id,d3_id,d4_id,d5_id,d6_id,g1_id,g2_id
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,8471817,8474157,8475166,8477503,8477939,8478483,8478904,8479318,8480144,8480980,8480995,8481582,8475171,8475690,8475825,8476853,8476931,8479026,8476932,8479361


In [17]:
def assign_player_gps_to_lineup_with_goalies(transposed_lineup_df, player_stats_df, goalie_stats_df):
    """
    Processes the transposed_lineup_df DataFrame to append games played (gp) next to each player's name.
    For skaters (Forwards and Defense), 'gp' is retrieved from player_stats_df.
    For goalies, 'gp' is retrieved from goalie_stats_df.
    
    This function performs the following steps:
        1. Iterates through each column in the lineup.
        2. Determines the player type based on the column prefix.
        3. Retrieves the corresponding 'gp' from the appropriate DataFrame.
        4. Inserts a new column adjacent to each player's name containing the 'gp' value.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        player_stats_df (pd.DataFrame): DataFrame containing skater statistics, including 'player' and 'gp'.
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics, including 'player' and 'gp'.
    
    Returns:
        pd.DataFrame: The updated DataFrame with 'gp' values added next to each player name.
    """
    import pandas as pd
    
    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")
    
    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()
    
    # Create mappings for 'gp' from player_stats_df and goalie_stats_df
    skater_gp_mapping = player_stats_df.set_index('player')['gp'].to_dict()
    goalie_gp_mapping = goalie_stats_df.set_index('player')['gp'].to_dict()
    
    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()
    
    # Iterate through each player column and add a new column with 'gp' value
    for col in player_columns:
        player_name = updated_df.at[0, col]
        gp_column = f"{col}_gp"
        
        if player_name != 'Empty':
            # Determine the player type based on column prefix
            if col.startswith(('f', 'd')):
                # Skater (Forward or Defense)
                gp_value = skater_gp_mapping.get(player_name, None)
                if gp_value is None:
                    print(f"Warning: 'gp' not found for skater '{player_name}'. Assigning None.")
            elif col.startswith('g'):
                # Goalie
                gp_value = goalie_gp_mapping.get(player_name, None)
                if gp_value is None:
                    print(f"Warning: 'gp' not found for goalie '{player_name}'. Assigning None.")
            else:
                # Unknown position prefix
                print(f"Warning: Column '{col}' does not start with F, D, or G. Unable to determine player type.")
                gp_value = None
            
            updated_df[gp_column] = gp_value
        else:
            updated_df[gp_column] = None  # Assign None for empty slots
    
    return updated_df

In [18]:
# Assuming you have already created `transposed_lineup_df`, `player_stats_df`, and `goalie_stats_df`

# Assign games played (gp) to both skaters and goalies
updated_lineup_with_gp_df = assign_player_gps_to_lineup_with_goalies(
    transposed_lineup_df,
    player_stats_df,
    goalie_stats_df
)

# Display the updated DataFrame
updated_lineup_with_gp_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_gp,f2_gp,f3_gp,f4_gp,f5_gp,f6_gp,f7_gp,f8_gp,f9_gp,f10_gp,f11_gp,f12_gp,d1_gp,d2_gp,d3_gp,d4_gp,d5_gp,d6_gp,g1_gp,g2_gp
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,16,13,18,19,19,19,19,13,18,3,17,16,19,19,2,19,19,1,12,5


In [19]:
# player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head() 

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,20,388.516667,1,3,1,2,4,20.00,17,5.88,0.77,50,28,11,2,3,5,12,6,6,0,0,1,22,4,11,14,22,0,0,-
1,Brent Burns,CAR,D,17,277.033333,0,5,2,3,5,33.33,26,0.00,1.12,69,42,19,2,1,5,4,2,2,0,0,1,24,7,2,8,14,0,0,-
2,Corey Perry,EDM,R,20,204.600000,3,0,0,0,3,42.86,18,16.67,2.20,38,26,22,11,0,1,14,4,2,2,0,7,6,3,5,10,5,0,2,0.00
3,Alex Ovechkin,WSH,L,18,215.250000,9,7,4,3,16,72.73,34,26.47,3.27,69,49,40,16,0,5,6,3,3,0,0,0,7,4,41,8,3,0,0,-
4,Evgeni Malkin,PIT,C,21,316.166667,3,9,6,3,12,75.00,30,10.00,4.62,61,40,36,20,0,8,8,4,4,0,0,1,20,3,9,18,16,62,80,43.66


In [21]:
yesterday

'2024-11-18'

In [32]:
from team_utils import get_week_schedule

schedule = get_week_schedule('TOR', '2024-11-10')
schedule

{'previousStartDate': '2024-11-03',
 'nextStartDate': '2024-11-17',
 'calendarUrl': 'https://nhl.ecal.com/maple-leafs',
 'clubTimezone': 'America/Toronto',
 'clubUTCOffset': '-05:00',
 'games': [{'id': 2024020245,
   'season': 20242025,
   'gameType': 2,
   'gameDate': '2024-11-12',
   'venue': {'default': 'Scotiabank Arena'},
   'neutralSite': False,
   'startTimeUTC': '2024-11-13T00:00:00Z',
   'easternUTCOffset': '-05:00',
   'venueUTCOffset': '-05:00',
   'venueTimezone': 'America/Toronto',
   'gameState': 'OFF',
   'gameScheduleState': 'OK',
   'tvBroadcasts': [{'id': 293,
     'market': 'H',
     'countryCode': 'CA',
     'network': 'TSN4',
     'sequenceNumber': 137},
    {'id': 33,
     'market': 'A',
     'countryCode': 'CA',
     'network': 'RDS',
     'sequenceNumber': 132},
    {'id': 294,
     'market': 'A',
     'countryCode': 'CA',
     'network': 'TSN5',
     'sequenceNumber': 135}],
   'awayTeam': {'id': 9,
    'commonName': {'default': 'Senators', 'fr': 'Sénateurs'},


In [27]:
def get_most_recent_game_id(schedule: Dict, reference_date: Optional[str] = None) -> Optional[int]:
    """
    Finds the most recent past game_id based on the game_date from the provided schedule.

    Args:
        schedule (dict): The schedule JSON data as returned by get_week_schedule.
        reference_date (str, optional): The date to compare against in 'YYYY-MM-DD' format.
                                        Defaults to today's date.

    Returns:
        Optional[int]: The game_id of the most recent past game. Returns None if no past games are found.
    """

    # Set reference_date to today if not provided
    if reference_date:
        try:
            ref_date = datetime.strptime(reference_date, '%Y-%m-%d').date()
        except ValueError:
            raise ValueError("reference_date must be in 'YYYY-MM-DD' format.")
    else:
        ref_date = datetime.today().date()
    
    # Filter games that have a gameDate before the reference_date
    past_games = [
        game for game in schedule.get('games', [])
        if datetime.strptime(game.get('gameDate'), '%Y-%m-%d').date() < ref_date
    ]

    if not past_games:
        return None

    # Find the game with the latest gameDate
    most_recent_game = max(past_games, key=lambda x: datetime.strptime(x.get('gameDate'), '%Y-%m-%d').date())

    return most_recent_game.get('id')

In [41]:
get_most_recent_game_id(schedule, '2024-11-19')

2024020274

In [36]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020274)

In [37]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)
away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8478585,10,D. Ryan,C,0,0,0,0,0,0,0,0,1.000000,08:47,0,14,0,1,Away
1,8477406,13,M. Janmark,C,0,1,1,1,0,1,0,0,0.000000,14:05,2,18,0,0,Away
2,8475786,18,Z. Hyman,L,0,1,1,-1,2,0,0,2,0.000000,21:52,0,22,1,0,Away
3,8474641,19,A. Henrique,C,1,0,1,1,0,0,0,2,0.777778,16:33,1,22,0,0,Away
4,8477015,28,C. Brown,R,0,1,1,1,0,1,0,1,0.000000,13:43,1,20,0,0,Away
5,8477934,29,L. Draisaitl,C,1,1,2,0,0,0,0,3,0.545455,22:09,1,23,0,0,Away
6,8475784,53,J. Skinner,L,0,0,0,-1,0,0,0,1,0.000000,11:10,1,15,1,0,Away
7,8470621,90,C. Perry,R,0,0,0,-1,0,0,0,1,1.000000,10:51,0,14,0,0,Away
8,8481617,92,V. Podkolzin,R,0,0,0,0,0,4,0,2,0.000000,12:10,1,16,0,0,Away
9,8476454,93,R. Nugent-Hopkins,C,0,1,1,-1,0,0,0,0,0.000000,20:57,1,21,1,0,Away


In [38]:
away_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,shotsAgainst,saves,team,savePctg,decision
0,8475717,30,C. Pickard,G,0/0,0/0,0/0,0/0,0,0,0,0,0,00:00,False,0,0,Away,NaN,NaN
1,8479973,74,S. Skinner,G,12/16,4/4,2/2,18/22,4,0,0,0,4,60:04,True,22,18,Away,0.818182,O


In [39]:
home_skaters

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8477503,11,M. Domi,C,0,0,0,0,0,0,0,1,0.333333,14:42,1,20,2,1,Home
1,8478483,16,M. Marner,R,1,1,2,3,0,0,0,3,0.000000,24:00,0,25,0,1,Home
2,8478904,18,S. Lorentz,C,0,0,0,0,0,0,0,0,0.000000,06:46,0,11,0,0,Home
3,8482720,23,M. Knies,L,1,0,1,0,0,1,0,1,0.000000,21:05,0,23,0,0,Home
4,8480980,24,C. Dewar,C,0,0,0,0,0,4,0,0,0.000000,08:40,0,14,1,0,Home
5,8480995,29,P. Holmberg,R,0,0,0,-1,0,0,0,0,0.000000,13:02,2,16,0,0,Home
6,8480144,64,D. Kampf,C,0,0,0,0,0,1,0,0,0.400000,14:22,1,19,1,0,Home
7,8482259,74,B. McMann,C,2,0,2,2,0,1,0,3,0.000000,19:06,2,23,0,0,Home
8,8471817,75,R. Reaves,R,0,0,0,0,15,1,0,0,0.000000,03:54,0,5,0,0,Home
9,8477939,88,W. Nylander,R,0,0,0,-1,0,0,0,4,0.571429,21:18,0,21,0,0,Home


In [40]:
home_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8476932,41,A. Stolarz,G,24/26,2/3,1/1,27/30,0.9,2,1,0,0,3,60:40,True,W,30,27,Home
1,8479361,60,J. Woll,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Home


In [17]:
lineup2.display_lineup()


Lineup: Lineup 2
Forwards:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
  Slot 7: Empty
  Slot 8: Empty
  Slot 9: Empty
  Slot 10: Empty
  Slot 11: Empty
  Slot 12: Empty
Defense:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
Goalie: Empty

